In [1]:
import matlab.engine
eng = matlab.engine.connect_matlab('primary')

In [2]:
#load data from MATLAB and run ANOVA across and trials for each channel
import numpy as np
import json
import scipy.stats as stats

catList = ['Target', 'Distractor', 'Irrelevant']

scale = np.squeeze(eng.workspace['scale'])
chanNum=np.squeeze(eng.workspace['chanNum'])
# chanNum = np.append(chanNum, 0)
chanLabel=eng.workspace['chanLabel']
# chAvg  = ['Channel Average']
# chanLabel.append(chAvg)
winWavelet = np.squeeze(eng.workspace['waveWin']).astype(int)
timeWavelet = np.linspace(-2,1,winWavelet.shape[0],endpoint=True)

winLFP = np.squeeze(eng.workspace['LFPWin']).astype(int)
timeLFP = np.linspace(-2,1,winLFP.shape[0],endpoint=True)

chanLabel_json = json.dumps(chanLabel)
chanNum_json = json.dumps(chanNum.tolist())

def run_stats(wavelet_data,LFP_data):
    num_rows = wavelet_data.shape[0]
    num_columns = wavelet_data.shape[1]
    num_channels = wavelet_data.shape[3]
    num_rows_LFP = LFP_data.shape[0]

    pvals_wavelet = np.empty((num_rows, num_columns, num_channels))
    pvals_LFP = np.empty((num_rows_LFP,num_channels))

    for chan in range(num_channels):
        group_wavelet = []
        group_LFP = []
        for categ in catList:
            wavelet_array = np.array(eng.eval(f'wavelet.{categ}'))
            group_wavelet.append(wavelet_array[..., chan])
            LFP_array = np.array(eng.eval(f'LFP.{categ}'))
            group_LFP.append(LFP_array[..., chan])
        for row in range(num_rows):
            for column in range(num_columns):
                f_stat_wavelet, p_val_wavelet = stats.f_oneway(group_wavelet[0][row, column], group_wavelet[1][row, column], group_wavelet[2][row, column])
                pvals_wavelet[row, column, chan] = p_val_wavelet
        for row in range(num_rows_LFP):
                f_stat_LFP, pval_LFP = stats.f_oneway(group_LFP[0][row], group_LFP[1][row], group_LFP[2][row])
                pvals_LFP[row,chan] = pval_LFP
    
    return pvals_wavelet,pvals_LFP

wavelet_dictionary = {}
LFP_dictionary = {}
pvals_wavelet,pvals_LFP = None,None
FirstIter=True


for categ in catList:
    #Load data from MATLAB    
    wavelet_data = np.array(eng.eval(f'wavelet.{categ}'))
    LFP_data = np.array(eng.eval(f'LFP.{categ}'))
    display(wavelet_data.shape)
    #average LFP/wavelet trials and channels as a slice in order to concatenate it to the spectrogram arrays as extra frames for animate, not extra htmls for create_template 
    # trial_avgs_wavelet = np.mean(wavelet_data, axis=2, keepdims=True).astype(np.uint16)
    # trial_avgs_LFP = np.mean(LFP_data, axis=1, keepdims=True)
    # chan_avgs_wavelet = np.mean(wavelet_data, axis=3, keepdims=True).astype(np.uint16)
    # chan_avgs_LFP = np.mean(LFP_data, axis=2, keepdims=True)
    # padding_wavelet = ((0, 0), (0, 0), (0, 2), (0, 0))
    # padding_LFP = ((0, 0), (0, 2), (0, 0))
    # chan_avgs_wavelet_padded=np.pad(chan_avgs_wavelet,pad_width=padding_wavelet,mode='constant',constant_values=0)
    # chan_avgs_LFP_padded=np.pad(chan_avgs_LFP,pad_width=padding_LFP,mode='constant',constant_values=0)
    # Run ANOVA between the three different category groups (ie Target,Distractor,Irrelevant) for each channel
    # cannot run ANOVA for each trial because trials are not the same number between category groups 
    # if FirstIter:
    #     pvals_wavelet,pvals_LFP= run_stats(wavelet_data,LFP_data)
    
    # #concatenate trial/channel averages and ANOVAs to the end of the spectrogram arrays
    # concat_wavelet_trials = np.concatenate((wavelet_data, trial_avgs_wavelet, np.expand_dims(pvals_wavelet, axis=2)), axis=2)
    # concat_LFP_trials = np.concatenate((LFP_data, trial_avgs_LFP,np.expand_dims(pvals_LFP, axis=1)), axis=1)
    # concat_wavelet = np.concatenate((concat_wavelet_trials,chan_avgs_wavelet_padded), axis=3)
    # concat_LFP = np.concatenate((concat_LFP_trials,chan_avgs_LFP_padded), axis=2)

    # wavelet_dictionary[categ] = concat_wavelet
    # LFP_dictionary[categ] = concat_LFP
    wavelet_dictionary[categ] = wavelet_data
    LFP_dictionary[categ] = LFP_data
    FirstIter=False


(28, 201, 131, 190)

(28, 201, 169, 190)

(28, 201, 116, 190)

In [5]:
import os
import numpy as np

np.save('timeWavelet.npy',timeWavelet)
np.save('winWavelet.npy',winWavelet)
np.save('scale.npy',scale)
for category, array in wavelet_dictionary.items():
    if not os.path.exists(category):
        os.mkdir(category)
    data = array
    display(data.shape)
    for channel in range(data.shape[-1]):
        channel_data = data[:, :, :, channel]
        # Save the numpy array with the key name in the current directory
        save_path = os.path.join(category, f'Channel{channel + 1}.npy')
        np.save(save_path, channel_data)

(28, 201, 131, 190)

(28, 201, 169, 190)

(28, 201, 116, 190)

In [ ]:
#convert numpy arrays to jsons
import numpy as np
import json
import os

class DataConverter:
    def __init__(self, wavelet_dictionary,winWavelet,timeWavelet,scale):
        self.data_dict = wavelet_dictionary
        self.winWavelet = winWavelet
        self.timeWavelet = timeWavelet
        self.scale = scale

    def _save_to_json(self, data, directory, trial):
        if not os.path.exists(directory):
            os.makedirs(directory)
        
        with open(f'{directory}/trial{trial+1}.json', 'w') as f:
            json.dump(data, f)

    def _convert_data_point(self, category_data, time_bin, freq_bin, trial, channel):
        return {
            "time": self.timeWavelet[time_bin].item(),
            "frequency": self.scale[freq_bin].item(),
            "power": category_data[freq_bin, time_bin, trial, channel].item()
        }
    
    def _structure_data_for_trial_and_channel(self, category_data, trial, channel):
        structured_data = [
            self._convert_data_point(category_data, time_bin, freq_bin, trial, channel)
            for time_bin in range(category_data.shape[1])
                for freq_bin in range(category_data.shape[0])
        ]
        return structured_data

    def convert_to_json(self):
        """
        Convert the numpy data in the dictionary to structured JSON.
        """
        for category, category_data in self.data_dict.items():
            for channel in range(category_data.shape[3]):
                for trial in range(category_data.shape[2]):
                    structured_data = self._structure_data_for_trial_and_channel(category_data[:,self.winWavelet,...], trial, channel)
                    directory = f'./{category}/channel{channel+1}'
                    self._save_to_json(structured_data, directory, trial)

converter = DataConverter(wavelet_dictionary,winWavelet,timeWavelet,scale)
converter.convert_to_json()


In [ ]:
# Create template (blank) htmls with embedded animations (OOP approach)
import numpy as np
from matplotlib import animation, colors, pyplot as plt
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import HTML
import os
import gc

class HeatmapAnimation:
    
    def __init__(self, wavelet_dictionary, LFP_dictionary, timeLFP, timeWavelet, winLFP, winWavelet, scale):
        self.wavelet_dictionary = wavelet_dictionary
        self.LFP_dictionary = LFP_dictionary
        self.timeLFP = timeLFP
        self.timeWavelet = timeWavelet
        self.winLFP = winLFP
        self.winWavelet = winWavelet
        self.scale = scale
        self.scale_bins = [np.arange(b[0], b[1]) for b in [(21,28),(13,21),(0,13)]]
        
    def set_heatmap(self, ax, im):
        ax.set_yscale('log')
        ax.set_ylabel('Frequency (Hz)', fontsize = 12)
        divider = make_axes_locatable(ax)
        cax = divider.append_axes('right', size='2%', pad='3%')
        cbar = plt.colorbar(im, cax)
        cbar.set_label('uV/Hz^2', fontsize=12)
        return cbar

    def set_figure(self):
        fig, axes = plt.subplots(4, 1, figsize=(15,10), sharex=True)
        fig.set_facecolor('#D3D3D3')
        axes[-1].set_xlabel('Time from response (sec)', fontsize = 12)
        axes[-1].set_ylabel('LFP (uV)', fontsize = 12)  
        divider = make_axes_locatable(axes[-1])
        cax = divider.append_axes('right', size='2%', pad='3%')
        cax.set_axis_off()
        return axes, fig

    def create_template(self, categ):
        waveletData = self.wavelet_dictionary[categ]
        LFPdata = self.LFP_dictionary[categ]
        
        self._process_channels(categ, waveletData, LFPdata)
        self._process_trials(categ, waveletData, LFPdata)
    
    def animate(self,trial,chan,im1,im2,axes,cb,waveletData,LFPdata):
                im1.set_data(timeLFP,LFPdata[winLFP,trial,chan])
                for ax, scale_bin in zip(range(3),self.scale_bins):
                    im2[ax].set_array(waveletData[scale_bin[:,np.newaxis],winWavelet,trial,chan])
                    if trial == waveletData.shape[2]-2:
                        LFPstd=np.std(LFPdata[winLFP,trial,chan])
                        axes[-1].set_ylim(-3*LFPstd,3*LFPstd)
                        wavelet_std = np.std(waveletData[scale_bin[:,np.newaxis],winWavelet,trial,chan])
                        im2[ax].set_clim(1,3 * wavelet_std)
                    if trial == waveletData.shape[2]-1:
                        axes[-1].set_ylim(0,1)
                        axes[-1].set_ylabel('p-Values', fontsize = 12)
                        im2[ax].set_clim(0,.1)
                        im2[ax].set_cmap('viridis_r')
                        cb[ax].set_label('p-Values', fontsize=12)
                    if chan == waveletData.shape[3]-1:
                        LFPstd=np.std(LFPdata[winLFP,trial,chan])
                        axes[-1].set_ylim(3*-LFPstd,3*LFPstd)
                        wavelet_std = np.std(waveletData[scale_bin[:,np.newaxis],winWavelet,trial,chan])
                        im2[ax].set_clim(1, 3 * wavelet_std)

    def _process_channels(self, categ, waveletData, LFPdata):
        for chan in range(waveletData.shape[3]): 
        # for chan in range(10,20):
            axes,fig=self.set_figure()
            im2=[]
            cb=[]
            im1,=axes[-1].plot(timeLFP,LFPdata[winLFP,0,chan])  
            LFPstd=np.std(LFPdata[winLFP,:-2,chan])
            axes[-1].set_ylim(-3*LFPstd,3*LFPstd)
            
            for ax, scale_bin in zip(axes[:-1],self.scale_bins):
                vmin = 1
                vmax = 3 * np.std(waveletData[scale_bin[:,np.newaxis],winWavelet,:-2,chan])
                scale_slice = waveletData[scale_bin[:,np.newaxis],winWavelet,0,chan]
                im = ax.pcolor(timeWavelet, scale[scale_bin], scale_slice, norm=colors.Normalize(vmin, vmax))
                cbar=self.set_heatmap(ax,im)
                im2.append(im)
                cb.append(cbar)
                               
            plt.tight_layout
            plt.close()
            z = lambda trial: self.animate(trial,chan,im1,im2,axes,cb,waveletData,LFPdata)
            anim = animation.FuncAnimation(fig,z,frames=waveletData.shape[2])
            html = HTML(anim.to_jshtml())
            with open(f'{categ}/Channel {chan}.html', 'w') as f:
                f.write(html.data)

    def _process_trials(self, categ, waveletData, LFPdata):
        for trial in range(waveletData.shape[2]):
        # for trial in range(5,10):
            axes,fig=self.set_figure()
            im2=[]
            cb=[]
            im1,=axes[-1].plot(timeLFP,LFPdata[winLFP,trial,0])  
            LFPstd=np.std(LFPdata[winLFP,trial,:-1])
            axes[-1].set_ylim(3*-LFPstd,3*LFPstd)
            
            for ax, scale_bin in zip(axes[:-1],self.scale_bins):
                vmin = 1
                vmax = 3 * np.std(waveletData[scale_bin[:,np.newaxis],winWavelet,trial,:-1])
                scale_slice = waveletData[scale_bin[:,np.newaxis],winWavelet,trial,0]
                im = ax.pcolor(timeWavelet, scale[scale_bin], scale_slice, norm=colors.Normalize(vmin, vmax))
                cbar=self.set_heatmap(ax,im)
                im2.append(im)
                cb.append(cbar)
                            
            def animate(chan):
                im1.set_data(timeLFP,LFPdata[winLFP,trial,chan])
                for ax, scale_bin in zip(range(3),self.scale_bins):
                    im2[ax].set_array(waveletData[scale_bin[:,np.newaxis],winWavelet,trial,chan])
                    
            plt.tight_layout
            plt.close()
            z2 = lambda chan: self.animate(trial,chan,im1,im2,axes,cb,waveletData,LFPdata)
            anim = animation.FuncAnimation(fig,z2,frames=waveletData.shape[3])

            html = HTML(anim.to_jshtml())
            with open(f'{categ}/Trial {trial}.html', 'w') as f:
                f.write(html.data)
        
    def generate(self, catList):
        for categ in catList:
            os.makedirs(categ, exist_ok=True)
            self.create_template(categ)
            gc.collect()

animator = HeatmapAnimation(wavelet_dictionary, LFP_dictionary, timeLFP, timeWavelet, winLFP, winWavelet, scale)
animator.generate(catList)


In [ ]:
# Create template (blank) htmls with embedded animations
import gc
gc.collect()
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 2**128
from matplotlib import animation, colors, pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import HTML

scale_bins=[np.arange(b[0], b[1]) for b in [(21,28),(13,21),(0,13)]]

def set_heatmap(ax,im):
    ax.set_yscale('log')
    ax.set_ylabel('Frequency (Hz)', fontsize = 12)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right',size='2%',pad='3%')
    cbar=plt.colorbar(im, cax)
    cbar.set_label('uV/Hz^2', fontsize=12) # For raw spectrograms
    #cbar.set_label('Z-Score', fontsize=12)  # For PCA reconstructed spectrograms
    return cbar
        
def set_figure():
    fig, axes = plt.subplots(4,1, figsize=(15,10),sharex=True)
    fig.set_facecolor('#D3D3D3')
    axes[-1].set_xlabel('Time from response (sec)', fontsize = 12)
    axes[-1].set_ylabel('LFP (uV)', fontsize = 12)  
    divider = make_axes_locatable(axes[-1])
    cax = divider.append_axes('right',size='2%',pad='3%')
    cax.set_axis_off()
    return axes, fig

def create_template(categ):
    waveletData = wavelet_dictionary[categ]
    LFPdata = LFP_dictionary[categ]
  
    for chan in range(waveletData.shape[3]): 
    # for chan in range(10,20):
        axes,fig=set_figure()
        im2=[]
        cb=[]
        im1,=axes[-1].plot(timeLFP,LFPdata[winLFP,0,chan])  
        LFPstd=np.std(LFPdata[winLFP,:-2,chan])
        axes[-1].set_ylim(3*-LFPstd,3*LFPstd)
        
        for ax, scale_bin in zip(axes[:-1],scale_bins):
            vmin = 1
            vmax = 3 * np.std(waveletData[scale_bin[:,np.newaxis],winWavelet,:-2,chan])
            scale_slice = waveletData[scale_bin[:,np.newaxis],winWavelet,0,chan]
            im = ax.pcolor(timeWavelet, scale[scale_bin], scale_slice, norm=colors.Normalize(vmin, vmax))
            cbar=set_heatmap(ax,im)
            im2.append(im)
            cb.append(cbar)

            
        def animate(trial):
            im1.set_data(timeLFP,LFPdata[winLFP,trial,chan])
            for ax, scale_bin in zip(range(3),scale_bins):
                im2[ax].set_array(waveletData[scale_bin[:,np.newaxis],winWavelet,trial,chan])
                if trial == waveletData.shape[2]-2:
                    LFPstd=np.std(LFPdata[winLFP,trial,chan])
                    axes[-1].set_ylim(3*-LFPstd,3*LFPstd)
                    wavelet_std = np.std(waveletData[scale_bin[:,np.newaxis],winWavelet,trial,chan])
                    im2[ax].set_clim(1,3 * wavelet_std)
                if trial == waveletData.shape[2]-1:
                    axes[-1].set_ylim(0,1)
                    axes[-1].set_ylabel('p-Values', fontsize = 12)
                    im2[ax].set_clim(0,.1)
                    im2[ax].set_cmap('viridis_r')
                    cb[ax].set_label('p-Values', fontsize=12)
                    
        plt.tight_layout
        plt.close()
        anim = animation.FuncAnimation(fig,animate,frames=waveletData.shape[2])
        html = HTML(anim.to_jshtml())
        
        with open(f'{categ}/Channel {chan}.html', 'w') as f:
            f.write(html.data)
    for trial in range(waveletData.shape[2]):
    # for trial in range(5,10):
        axes,fig=set_figure()
        im2=[]
        im1,=axes[-1].plot(timeLFP,LFPdata[winLFP,trial,0])  
        LFPstd=np.std(LFPdata[winLFP,trial,:-1])
        axes[-1].set_ylim(3*-LFPstd,3*LFPstd)
        
        for ax, scale_bin in zip(axes[:-1],scale_bins):
            vmin = 1
            vmax = 3 * np.std(waveletData[scale_bin[:,np.newaxis],winWavelet,trial,:-1])
            scale_slice = waveletData[scale_bin[:,np.newaxis],winWavelet,trial,0]
            im = ax.pcolor(timeWavelet, scale[scale_bin], scale_slice, norm=colors.Normalize(vmin, vmax))
            cbar=set_heatmap(ax,im)
            im2.append(im)
                        
        def animate(chan):
            im1.set_data(timeLFP,LFPdata[winLFP,trial,chan])
            for ax, scale_bin in zip(range(3),scale_bins):
                im2[ax].set_array(waveletData[scale_bin[:,np.newaxis],winWavelet,trial,chan])
                if chan == waveletData.shape[3]-1:
                    LFPstd=np.std(LFPdata[winLFP,trial,chan])
                    axes[-1].set_ylim(3*-LFPstd,3*LFPstd)
                    wavelet_std = np.std(waveletData[scale_bin[:,np.newaxis],winWavelet,trial,chan])
                    im2[ax].set_clim(1,3 * wavelet_std)

        plt.tight_layout
        plt.close()
        anim = animation.FuncAnimation(fig,animate,frames=waveletData.shape[3])
        html = HTML(anim.to_jshtml())
        with open(f'{categ}/Trial {trial}.html', 'w') as f:
            f.write(html.data)

for categ in catList:
    os.makedirs(categ, exist_ok=True)
    create_template(categ)
    gc.collect()

# create_template("Target")
# gc.collect() 

In [ ]:
#html design
import bs4
catList = ["Target","Distractor","Irrelevant"]
trialScroll=False

for categ in catList:
    if trialScroll:
        size = wavelet_dictionary[categ].shape[3]
    else:
        size = wavelet_dictionary[categ].shape[2]
    
    for i in range(size):
    # for i in range(10):    
        if trialScroll:
            fileFolder = f'{categ}/Channel {i}.html'
            chLabel = chanLabel[i][0]
            chNum = str(int(chanNum[i]))
            pageTitle = f'CH#{chNum}   {chLabel}'
        else:
            fileFolder = f'{categ}/Trial {i}.html'
            if i == size-1:
                pageTitle = 'ANOVA'
            elif i == size-2:
                pageTitle = 'Trial Average'
            else:
                pageTitle = f'Trial {i+1}'
                
        with open(fileFolder) as inf:    
            txt = inf.read()
        
        soup = bs4.BeautifulSoup(txt)
        
        home_button = soup.new_tag('a', href='../../../index.html')
        home_button['style'] = 'color: blue; text-decoration: underline; cursor: pointer; font-size: 20px; position: absolute; left: 25%; top: 1%;'
        home_button.string = "Return to Home"

        script_element = soup.find("script", language="javascript")
        js_code = script_element.string.replace("this.current_frame = 0", "this.current_frame=initialFrame")
        script_element.string = js_code

        new_js_code = """
        const urlParams = new URLSearchParams(window.location.search);
        const initialFrame = urlParams.get("frame");
        """ + script_element.string

        # Replace the content of the <script> element with the updated code
        script_element.string = new_js_code
        
        newBody = soup.new_tag("body", style="background-color: #D3D3D3;")
        pageHead = soup.new_tag("h1", style="display: flex; justify-content: center; margin-top: 35px;")
        pageHead.append(pageTitle)

        animation_div = soup.find("div", class_="animation")
        animation_div["style"] = "display: block;"
        
        animation_div.insert_before(newBody)
        animation_div.insert_before(home_button)
        animation_div.insert_before(pageHead)
                
        script_tag = soup.new_tag('script')
        js_code = f"""
        var chanLabel = {chanLabel_json};
        var chanNum = {chanNum_json};
        """
        script_tag.string = js_code
        animation_div.insert_before(script_tag)
        
        button_links_div = soup.new_tag("div", style="display: flex; justify-content: center; gap: 100px;")
        for button_text in catList:
            if button_text == categ: 
                categ_button = soup.new_tag("span", id=f"{button_text}Button")
                categ_button['disabled'] = 'true'
            elif trialScroll:
                link = f"../{button_text}/Channel {i}.html"
                categ_button = soup.new_tag("button")
                categ_button["onclick"] = f"location.href='{link}?frame=0';"
            else:
                categ_button = soup.new_tag('button', id=f"{button_text}Button")
            categ_button.string = button_text        
            button_links_div.append(categ_button)
        animation_div.insert_before(button_links_div)

       
        button_div = soup.new_tag("div", style="display: flex; justify-content: center; gap:50px;")
        if trialScroll:
            if i != 0:
                prev_chan_button = soup.new_tag('button', id="prevChanButton") 
                prev_chan_button.string = '< Previous Channel'
                button_div.append(prev_chan_button)
            if i != size-1:
                next_chan_button = soup.new_tag('button', id="nextChanButton")
                next_chan_button.string = 'Next Channel >'
                button_div.append(next_chan_button)
        else:
            if i != 0:
                prev_trial_button = soup.new_tag('button', id="prevTrialButton")
                prev_trial_button.string = '< Previous Trial'
                button_div.append(prev_trial_button)
            if i != size-1:
                next_trial_button = soup.new_tag('button', id="nextTrialButton")
                next_trial_button.string = 'Next Trial >'
                button_div.append(next_trial_button)
        animation_div.insert_before(button_div)

        switch_button = soup.new_tag("a",  id="switchButton")
        switch_button['style'] = 'color: blue; text-decoration: underline; cursor: pointer; font-size: 20px; display: flex; justify-content: center;'
        animation_div.insert_after(switch_button)

        # display Trial or Channel# using Slider ID
        frame_display_element = soup.new_tag("p", id="frameDisplay", style='font-size: 24px; font-weight: bold; display: flex; justify-content: center;')
        animation_div.insert_after(frame_display_element)

        script_element = soup.find("script", string=lambda s: "Animation.prototype.set_frame" in s)
        if trialScroll:
            switch_button.string = "Scroll by Channel"
            script_element.string = script_element.string.replace(
                "document.getElementById(this.slider_id).value = this.current_frame;",
                """
                document.getElementById(this.slider_id).value = this.current_frame;
                var currentFrame = parseInt(this.current_frame);
                currentFrame += 1;
                var lastFrame = {}; 
                var secondLastFrame = {};
                
                if (currentFrame === lastFrame) {{
                    document.getElementById('frameDisplay').textContent = 'ANOVA';
                }} else if (currentFrame === secondLastFrame) {{
                    document.getElementById('frameDisplay').textContent = 'Trial Average';
                }} else {{
                    document.getElementById('frameDisplay').textContent = 'Trial ' + currentFrame;
                }}

                document.getElementById("switchButton").addEventListener("click", () => {{
                    const url = 'Trial ' + (this.current_frame) + '.html?frame={}';
                    window.location.href = url;
                }});
                
                """.format(wavelet_dictionary[categ].shape[2],(wavelet_dictionary[categ].shape[2])-1,i)
            )

            if i != 0:
                script_element.string = script_element.string.replace(
                    "document.getElementById(this.slider_id).value = this.current_frame;",
                    """
                    document.getElementById(this.slider_id).value = this.current_frame;
                    document.getElementById("prevChanButton").addEventListener("click", () => {{
                    const url = 'Channel {}.html?frame=' + this.current_frame;
                    window.location.href = url;
                    }});
                    """.format(i-1)
                )
            if i != size-1:
                script_element.string = script_element.string.replace(
                    "document.getElementById(this.slider_id).value = this.current_frame;",
                    """
                    document.getElementById(this.slider_id).value = this.current_frame;
                    document.getElementById("nextChanButton").addEventListener("click", () => {{
                    const url = 'Channel {}.html?frame=' + this.current_frame;
                    window.location.href = url;
                    }});
                    """.format(i+1)
                )

        else:
            switch_button.string = "Scroll by Trial"    
            script_element.string = script_element.string.replace(
                "document.getElementById(this.slider_id).value = this.current_frame;",
                """
                document.getElementById(this.slider_id).value = this.current_frame;
                var currentFrame = parseInt(this.current_frame);
                currentFrame += 1;
                var lastFrame = {}; 

                if (currentFrame === lastFrame) {{
                    document.getElementById('frameDisplay').textContent = 'Channel Average';
                }} else {{
                    document.getElementById("frameDisplay").textContent = "CH#" + chanNum[this.current_frame] + " " + chanLabel[this.current_frame];
                }}
                
                document.getElementById("switchButton").addEventListener("click", () => {{
                const url = 'Channel ' + (this.current_frame) + '.html?frame={}';
                window.location.href = url;
                }});
                
                document.getElementById("TargetButton").addEventListener("click", () => {{
                const categ_url = '../Target/Trial 0.html?frame=' + (this.current_frame);
                window.location.href = categ_url;
                }});

                document.getElementById("DistractorButton").addEventListener("click", () => {{
                const categ_url = '../Distractor/Trial 0.html?frame=' + (this.current_frame);
                window.location.href = categ_url;
                }});   
                
                document.getElementById("IrrelevantButton").addEventListener("click", () => {{
                const categ_url = '../Irrelevant/Trial 0.html?frame=' + (this.current_frame);
                window.location.href = categ_url;
                }});

                """.format(wavelet_dictionary[categ].shape[3],i)
            )         
            if i != 0:
                script_element.string = script_element.string.replace(
                    "document.getElementById(this.slider_id).value = this.current_frame;",
                    """
                    document.getElementById(this.slider_id).value = this.current_frame;
                    document.getElementById("prevTrialButton").addEventListener("click", () => {{
                    const url = 'Trial {}.html?frame=' + this.current_frame;
                    window.location.href = url;
                    }});
                    """.format(i-1)
            )
            if i != size-1: 
                script_element.string = script_element.string.replace(
                    "document.getElementById(this.slider_id).value = this.current_frame;",
                    """
                    document.getElementById(this.slider_id).value = this.current_frame;
                    document.getElementById("nextTrialButton").addEventListener("click", () => {{
                    const url = 'Trial {}.html?frame=' + this.current_frame;
                    window.location.href = url;
                    }});
                    """.format(i+1)
            )                        

        with open(fileFolder, "w") as outf:
            outf.write(str(soup))        

In [ ]:
# Create index of htmls files
subfolders = ["YDX Session1"]

main_folders = ["Target Status"]

# Template URL
template_url = f"{catList[0]}/Channel 0.html?frame=0"

# Initialize the HTML content
html_content = """<!DOCTYPE html>
<html>
<head>
    <title>Home Page</title>
    <style>
        ul {{
            text-align: center;
            padding: 50px;
        }}
     
        h1 {{
            text-align: center; 
        }}

        ul button {{
            margin-right: 100px; 
        }}
    </style>
</head>
<body>
    <h1>Spectrograms</h1>
    
    <ul>
        {}
    </ul>
</body>
</html>"""

# Generate links and buttons for the subfolders and their associated labels
links = []

for label in subfolders:
    # Create a list item with plain text label
    link_item = f'<ul>{label}: '

    # Add buttons for the main subfolders
    for main_folder in main_folders:
        # Generate the link for each subfolder within each main subfolder
        link = f"{main_folder}/{label}/{template_url}"
        # Add a button with the associated label and link
        link_item += f'<button onclick="location.href=\'{link}\';">{main_folder}</button> '

    # Close the list item
    link_item += '</ul>'

    # Add the list item to the list of links
    links.append(link_item)

# Write the content to a new HTML file
with open("index.html", "w") as file:
    file.write(html_content.format("\n".join(links)))

display('Operation complete')    


In [ ]:
# When needed: move htmls from template folder to main after deleting same folders from main 
import os
import shutil
catList = ["Target","Distractor","Irrelevant"]

# Get the current directory
current_dir = os.getcwd()

# Delete the subfolders to be deleted
for subfolder in catList:
    subfolder_path = os.path.join(current_dir, subfolder)
    if os.path.exists(subfolder_path):
        shutil.rmtree(subfolder_path)

# Copy contents of the 'template' subfolder to the current directory
template_subfolder = os.path.join(current_dir, "template")
if os.path.exists(template_subfolder):
    for item in os.listdir(template_subfolder):
        item_path = os.path.join(template_subfolder, item)
        if os.path.isdir(item_path):
            destination_path = os.path.join(current_dir, item)
            shutil.copytree(item_path, destination_path)
        else:
            shutil.copy2(item_path, current_dir)

print("Operation completed")
